In [ ]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt


# Example 4.3 (Timoshenko beam in 2d: the Reissner Mindlin plate)

Solve the Reissner Mindlin plate equation in NGSolve. This is the 2-dimensional version of the Timoshenko beam. 

Find a $(w, \beta) \in V \subset\left[H^{1}(\Omega)\right]^{3}\left(w \text { is scalar and } \beta=\left(\beta_{x}, \beta_{y}\right)\right.$ vector valued) such that
$$
\int_{\Omega} \nabla \beta \cdot \nabla \delta \mathrm{d} x+\frac{1}{t^{2}} \int_{\Omega}(\nabla w-\beta) \cdot(\nabla v-\delta) \mathrm{d} x=\int_{\Omega} f v \mathrm{d} x \quad \forall(v, \delta) \in V
$$
Use $\Omega=(0,1)^{2}, f=1$ and $t=0.1, t=0.01$ or $t=0.001$ and use the boundary conditions

a.) $w=\beta_{x}=\beta_{y}=0$ on $\partial \Omega$

b.) $w=\beta_{x}=\beta_{y}=0$ on the left boundary $\Gamma_{\text {left }} \cup \Gamma_{\text {top }}=\{\{0\} \times(0,1)\} \cup\{(0,1) \times\{1\}\}$

c) $w=0$ on $\partial \Omega$

Visualize $w$ and the components of $\beta .$ Show that $\nabla w-\beta \rightarrow 0$ in $L^{2}$ for $t \rightarrow 0 .$ 

For the implementation use again product spaces (compound FESpaces) as in the previous example. Try polynomial orders (for the approximations) $k=1,2,3$

In [ ]:
print("Welcome, Peter!")
### Definition of mesh ###
h = 0.1
geo = SplineGeometry()

geo.AddRectangle((0,0), (1,1), bcs=['b','r','t','l'], leftdomain=1, rightdomain=0)
geo.SetMaterial(1, "D")
mesh = Mesh(geo.GenerateMesh(maxh=h)) # standard mesh available
print('#'*40)
print("Mesh generated:")
print('#'*40)
print("Number of vertices: ", mesh.nv)
print("Number of elements: ", mesh.ne)
print("Dimension of mesh: ", mesh.dim)
Draw(mesh)

In [ ]:
### Definition of FES ###
# a)
BND = 'b|r|t|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetrix=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #test = Integrate(Norm(grad(w_sol) - beta_sol), mesh)
        print(test)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B1')
Draw(beta_sol, mesh, 'beta_B1')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()

In [ ]:
### Definition of FES ###
# b)
BND_b = 't|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetric=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B2')
Draw(beta_sol, mesh, 'beta_B2')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()

In [ ]:
### Definition of FES ###
# c)
BND_c = 'b|r|t|l'
orders = [1,2,3]
ts = [0.1, 0.01, 0.001]
fc = 1
Norms_all = [[],[],[]]

for order, Norms in zip(orders, Norms_all):
    Vw = H1(mesh, order = order, dirichlet=BND)
    Vb = VectorH1(mesh, order = order, dirichlet=BND)

    #This defines a "compound FESpace"
    V = FESpace([Vw, Vb])
    for t in ts:
        t2 = t*t
        # Trial and test functions
        (w, beta) = V.TrialFunction()
        (v, delta) = V.TestFunction()

        #help(beta)

        # BLF
        B = BilinearForm(V, symmetrix=True)
        B += (InnerProduct(grad(beta), grad(delta)) + 1/t2 * InnerProduct(grad(w) - beta, grad(v)-delta) )*dx
        # LF
        F = LinearForm(V)
        F += fc*v*dx

        # solution on V
        sol = GridFunction(V, 'sol_mixed')
        w_sol = sol.components[0]
        beta_sol = sol.components[1]

        ### Solving the system ###
        # Assemble the system of equations
        with TaskManager(): # with Multithreading
            F.Assemble()
            B.Assemble()

        # Solve the system
        #sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = "umfpack") * F.vec
        sol.vec.data = B.mat.Inverse(V.FreeDofs(), inverse = 'sparsecholesky') * F.vec

        ### Calculating stuff in the system ###

        # L2-norm of grad(w_sol) - beta_sol ... don't know how to do it directly with ngsolve
        norm = Integrate(InnerProduct(grad(w_sol) - beta_sol, grad(w_sol) - beta_sol), mesh)
        Norms.append(norm)
        #print("Norm(grad(w) - beta)=", norm)
        #print('-'*30)
#-------------------------------------------------------------------------------------------#

print("Trying to draw what you cobbled together here...")
Draw(w_sol, mesh, 'w_B3')
Draw(beta_sol, mesh, 'beta_B3')

fig, ax = plt.subplots(1, figsize= [10,6])
ax.loglog(ts, Norms_all[0], 'x-r', label='k=1')
ax.loglog(ts, Norms_all[1], 'x-b', label='k=2')
ax.loglog(ts, Norms_all[2], 'x-g', label='k=3')
ax.set_xlabel('t')
ax.set_ylabel('$||grad(\omega) - \\beta||_{L2}$')
ax.grid(True)
fig.show()